In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
sys.path.append('..')
from scipy.constants import h

# Single transmon qubit

## Create linear circuit

In [2]:
from tlsim2.lumped import LumpedTwoTerminal
from tlsim2.lumped import JosephsonJunction

C_q = 73e-15
EJ = 16e9 * h

# qubit capacitor
capacitor_qubit = LumpedTwoTerminal(name='C_q', c=C_q)

# qubit nonlinear element
jj =  JosephsonJunction(name='JJ', ej=EJ)

from tlsim2.circuit import Circuit

linear_circuit = Circuit()
linear_circuit.add_element(capacitor_qubit, {'i': 0, 'o': 1})
linear_circuit.add_element(jj, {'i': 0, 'o': 1})
linear_circuit.short(0)

w, v, node_names = linear_circuit.compute_system_modes()
decays = np.real(w/1e9)*2
frequencies = np.imag(w/(2*np.pi))/1e9
decays = np.real(w/1e9)*2
decays = decays[frequencies > 0]
frequencies = frequencies[frequencies > 0]
frequencies, decays 

(array([5.8278845]), array([0.]))

## Create nonlinear circuit

In [3]:
print(linear_circuit.nonlinear_elements)

from tlsim2.quantum_circuit import QuantumCircuit
nonlinear_circuit = QuantumCircuit(linear_circuit)

print(f"Number of modes in the system is {nonlinear_circuit.num_modes} and \
number of nonlinaer elements is {nonlinear_circuit.num_nonlinear_elements}")

OrderedDict([('JJ', <tlsim2.lumped.JosephsonJunction object at 0x000002687FF79040>)])
Number of modes in the system is 1 and number of nonlinaer elements is 1


In [4]:
nonlinear_circuit.get_epr_coefficients(), np.sum(nonlinear_circuit.get_epr_coefficients())

(array([[1.]]), 0.9999999999999999)

In [5]:
kerr_matrix, anharmonicities = nonlinear_circuit.get_kerr_matrix()
kerr_matrix / (2 * np.pi), anharmonicities / (2 * np.pi)

(array([[5.30691214e+08]]), array([2.65345607e+08]))

Plasma frequency of transmon qubit in Hz
$$
\omega_{01} =  \sqrt{8 E_C E_J},
$$
where $E_C = [\text{Hz}]$ and $E_J = [\text{Hz}]$

Anharmonicity
$$
\alpha = -E_C
$$

In [6]:
from scipy.constants import hbar, e, h

# Qubit
EC = e ** 2 / (C_q) / 2 / h
f_q = np.sqrt(8 * EC * EJ / h)

print(f"Plasma frequency for transmon qubit is {f_q / 1e9} GHz and anharmonicity {- EC / 1e6} MHz")

Plasma frequency for transmon qubit is 5.827884497821679 GHz and anharmonicity -265.3456071871113 MHz


# Circuit in linear approximation

In [7]:
from tlsim2.tl import MultiTransmissionLine
from tlsim2.tl import default_tl_basis

from tlsim2.lumped import LumpedTwoTerminal
from tlsim2.lumped import JosephsonJunction

ll = [[416.120e-9]] # per-unit-length inductance for a 50-ohm TL on cold silicon
cl = [[166.448e-12]] # per-unit-length capacitance for a 50-ohm TL on cold silicon

vp = 1/np.sqrt(np.asarray(cl)*np.asarray(ll))
length = 4e-3

C_q = 73e-15
EJ = 16e9 * h
C_c = 4e-15

# resonator transmission line
default_basis1 = default_tl_basis(1, n_harmonics=2)
tl = MultiTransmissionLine('TL', n=1, l=length, ll=ll, cl=cl, basis=default_basis1)

# qubit capacitor
capacitor_qubit = LumpedTwoTerminal(name='C_q', c=C_q)

# qubit nonlinear element
jj =  JosephsonJunction(name='JJ', ej=EJ)

# coupling capacitor
capacitor_coupl = LumpedTwoTerminal(name='C_c', c=C_c)

In [8]:
from tlsim2.circuit import Circuit

linear_circuit = Circuit()
linear_circuit.add_element(capacitor_qubit, {'i': 0, 'o': 1})
linear_circuit.add_element(jj, {'i': 0, 'o': 1})
linear_circuit.add_element(tl, {'i0': 2, 'o0': 0})
linear_circuit.add_element(capacitor_coupl, {'i': 1, 'o': 2})
linear_circuit.short(0)

w, v, node_names = linear_circuit.compute_system_modes()
decays = np.real(w/1e9)*2
frequencies = np.imag(w/(2*np.pi))/1e9
decays = np.real(w/1e9)*2
decays = decays[frequencies > 0]
frequencies = frequencies[frequencies > 0]
frequencies, decays 

(array([102.23915691,  57.19206793,  37.60591848,  22.40341332,
          5.67185091,   7.47048357]),
 array([0., 0., 0., 0., 0., 0.]))

In [9]:
from scipy.constants import hbar, e, h

# Qubit
EC = e ** 2 / (C_q + C_c) / 2 / h
f_q = np.sqrt(8 * EC * EJ / h)

# Resonator frequnecy is changed due to 
f_r = 1 / (4 * length * np.sqrt(np.asarray(cl)*np.asarray(ll)))


f_q / 1e9, f_r[0, 0] / 1e9 # GHz

(5.674492200584848, 7.509852927040275)

# Nonlinear circuit

Check all nonlinear elements in the circuit

In [10]:
linear_circuit.nonlinear_elements

OrderedDict([('JJ', <tlsim2.lumped.JosephsonJunction at 0x26818137e50>)])

Now one can create quantum nonlinear circuit from this linear circuit

In [11]:
from tlsim2.quantum_circuit import QuantumCircuit
nonlinear_circuit = QuantumCircuit(linear_circuit)

print(f"Number of modes in the system is {nonlinear_circuit.num_modes} and \
number of nonlinaer elements is {nonlinear_circuit.num_nonlinear_elements}")

Number of modes in the system is 6 and number of nonlinaer elements is 1


In [12]:
nonlinear_circuit.get_epr_coefficients()

array([[1.94786718e-05],
       [6.55149253e-06],
       [1.64344368e-05],
       [4.54496244e-05],
       [9.97914233e-01],
       [1.99785267e-03]])

In [13]:
np.sum(nonlinear_circuit.get_epr_coefficients())

1.000000000000001

In [15]:
kerr_matrix, anharmonicities = nonlinear_circuit.get_kerr_matrix()
kerr_matrix / (2 * np.pi), anharmonicities / (2 * np.pi)

(array([[6.19688196e+01, 1.16593053e+01, 1.92312561e+01, 3.16840809e+01,
         1.76122422e+05, 4.64417737e+02],
        [1.16593053e+01, 2.19367420e+00, 3.61832108e+00, 5.96129434e+00,
         3.31370696e+04, 8.73792373e+01],
        [1.92312561e+01, 3.61832108e+00, 5.96818228e+00, 9.83276233e+00,
         5.46574135e+04, 1.44126296e+02],
        [3.16840809e+01, 5.96129434e+00, 9.83276233e+00, 1.61997758e+01,
         9.00497556e+04, 2.37452468e+02],
        [1.76122422e+05, 3.31370696e+04, 5.46574135e+04, 9.00497556e+04,
         5.00559920e+08, 1.31992794e+06],
        [4.64417737e+02, 8.73792373e+01, 1.44126296e+02, 2.37452468e+02,
         1.31992794e+06, 3.48052192e+03]]),
 array([3.09844098e+01, 1.09683710e+00, 2.98409114e+00, 8.09988791e+00,
        2.50279960e+08, 1.74026096e+03]))